In [1]:

## Libraries Prequisities
!pip install scikit-learn
!pip install numpy==1.16.1

In [0]:
Privacy-Preserving Collaborative Deep Learning with Application to Human
Activity Recognition

In [2]:
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics.classification import accuracy_score, recall_score, f1_score
import scipy.stats as st

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.2.0-rc2
/device:GPU:0
Num GPUs Available:  1


In [0]:
from tensorflow.keras.callbacks import Callback

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
np.random.seed(12227)

In [0]:
n_ep = 200
loss = 0.2
bs = 1000

In [0]:
def DataPreparation(data_input_file):
  print('Lingjuan Lyu  et al. 2017 {}'.format(data_input_file))
  data = np.load(data_input_file)
  X = data['X']
  X = X[:, 0, :, :]
  # X = X[:, 0, :, :]
  Y = data['y']
  folds = data['folds']
  # classes_number = Y.shape[1]
  # Y = np.argmax(Y, axis=1)
  return X,Y,folds

In [0]:
X,Y,f=DataPreparation('/content/drive/My Drive/Computer_vision_project/data/LOSO/MHEALTH.npz')

Bevilacqua  et al. 2015 /content/drive/My Drive/Computer_vision_project/data/LOSO/MHEALTH.npz


In [0]:
print(X[2].shape)

(1, 250, 23)


In [0]:
def ReportAccuracies(avg_acc, avg_recall,avg_f1):
  ic_acc = st.t.interval(0.9, len(avg_acc) - 1, loc=np.mean(avg_acc), scale=st.sem(avg_acc))
  ic_recall = st.t.interval(0.9, len(avg_recall) - 1, loc=np.mean(avg_recall), scale=st.sem(avg_recall))
  ic_f1 = st.t.interval(0.9, len(avg_f1) - 1, loc=np.mean(avg_f1), scale=st.sem(avg_f1))
  print('Mean Accuracy[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_acc), ic_acc[0], ic_acc[1]))
  print('Mean Recall[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_recall), ic_recall[0], ic_recall[1]))
  print('Mean F1[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_f1), ic_f1[0], ic_f1[1]))

In [0]:
def RunLingjuan_Lyu(data_input_file):
  X,Y,folds,= DataPreparation(data_input_file)
  avg_acc, avg_recall,avg_f1= Train(X,Y,folds)
  ReportAccuracies(avg_acc, avg_recall,avg_f1 )

In [0]:
def build_model(row,col,num_classes):

  tf.keras.initializers.GlorotNormal(234)
  input_layer = tf.keras.Input(shape=(row, col,1))
  
  layer = tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten())(input_layer)
  layer = tf.keras.layers.LSTM(28,return_sequences=True  )(layer)
  layer = tf.keras.layers.Dropout(0.2)(layer)
  layer = tf.keras.layers.LSTM(28,return_sequences=True)(layer)
  layer = tf.keras.layers.Dropout(0.2)(layer)
  layerconv1 = tf.keras.layers.Conv1D(filters=30,kernel_size=5,  strides=1) (layer)
  layerconv1 = tf.keras.layers.Activation('relu')(layerconv1)
  layerconv2 = tf.keras.layers.Conv1D(filters=40,kernel_size= 10,strides=1) (layer)
  layerconv2 = tf.keras.layers.Activation('relu')(layerconv2)
  layerconv3 = tf.keras.layers.Conv1D(filters=50,kernel_size= 15,strides=1) (layer)
  layerconv3 = tf.keras.layers.Activation('relu')(layerconv3)
  layerconv4 = tf.keras.layers.Conv1D(filters=60,kernel_size= 20,strides=1) (layer)
  layerconv4 = tf.keras.layers.Activation('relu')(layerconv4)
  max1= tf.reduce_max(layerconv1, 1)
  max2= tf.reduce_max(layerconv2, 1)
  max3= tf.reduce_max(layerconv3, 1)
  max4= tf.reduce_max(layerconv4, 1)
  concat_layer = tf.keras.layers.concatenate([max1,max2,max3,max4],1)
  layer= tf.keras.layers.Dense(num_classes, activation = 'softmax')   (concat_layer) 

    
 
  # layers = [
     
  #      tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()),
          
        
  #       tf.keras.layers.LSTM(28,return_sequences=True,  input_shape=(row, col)),
  #       tf.keras.layers.Dropout(0.2),
  #       tf.keras.layers.LSTM(64,return_sequences=True), 
  #       tf.keras.layers.Dropout(0.2),
  #       tf.keras.layers.Conv1D(filters=30,kernel_size=5,strides=1),
        
  #       tf.keras.layers.Conv1D(filters=40,kernel_size= 10,strides=1),
  #       tf.keras.layers.Conv1D(filters=50,kernel_size= 15,strides=1),
  #       tf.keras.layers.Conv1D(filters=60,kernel_size= 20,strides=1),
  #       tf.keras.layers.Dropout(0.2),

        
                       

       
        
       
  #       tf.keras.layers.Dense(num_classes, activation = 'softmax')
  #   ]
    
  model = tf.keras.Model(inputs=input_layer, outputs=layer)
  return model

In [0]:
# def build_model_1(row,col,num_classes):

 
 
#   layers = [
#         tf.keras.layers.Conv2D(filters=10,kernel_size=(3, 5), activation = tf.nn.relu),
#         tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.MaxPool2D(pool_size=(3,3),strides = 1),

#         tf.keras.layers.Conv2D(filters=2,kernel_size=(2, 4),activation = tf.nn.relu),
#         tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.MaxPool2D(pool_size=(2,2),strides = 1),

#         tf.keras.layers.Conv2D(filters=2,kernel_size=(2, 2),activation = tf.nn.relu),
#         tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.MaxPool2D(pool_size=(3,2),strides = 1),
        
        

       

       
#         tf.keras.layers.Flatten(),
#         tf.keras.layers.Dense(500, activation = tf.nn.relu),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.Dense(25, activation = tf.nn.relu),
#         tf.keras.layers.Dropout(0.5),
       
#         tf.keras.layers.Dense(num_classes, activation = 'softmax')
#     ]
    
#   model = tf.keras.Sequential(layers)
#   return model

In [0]:
batch_size = 128
epochs = 100
# validation_split = 0.15
# stopping_patience = 100
# model_save_frequency = 50
# # AdaDelta parameters
learning_rate = 0.001
# rho = 0.95
# epsilon = 1e-08


In [0]:
def Train(X,y,folds):
  avg_acc = []
  avg_recall = []
  avg_f1 = []
  n_class = y.shape[1]
  _, img_rows, img_cols = X.shape
  X=X.reshape(X.shape[0],img_rows,img_cols,1)
  _,img_rows, img_cols,_ = X.shape
  print(X.shape)
  for i in range(0, len(folds)):
    train_idx = folds[i][0]
    test_idx = folds[i][1]
    X_train = X[train_idx]
    X_test = X[test_idx]
    
    model=build_model(img_rows, img_cols, n_class)
    optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
    # callbacks = [
    #     tf.keras.callbacks.EarlyStopping(patience=stopping_patience, restore_best_weights=True),
        
    # ]

    model.compile(loss="categorical_crossentropy", optimizer=optimizer,  metrics=['accuracy'])
    # print(model.summary)
    
    model.fit(X_train, y[train_idx], batch_size=batch_size,epochs=epochs,verbos=2)
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y[test_idx], axis=1)
    acc_fold = accuracy_score(y_true, y_pred)
    avg_acc.append(acc_fold)
    recall_fold = recall_score(y_true, y_pred, average='macro')
    avg_recall.append(recall_fold)
    f1_fold = f1_score(y_true, y_pred, average='macro')
    avg_f1.append(f1_fold)
    print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}] at fold[{}]'.format(acc_fold, recall_fold, f1_fold ,i+1))
    print('________________________________________________________________')
    
  return avg_acc, avg_recall,avg_f1

In [0]:
tf.keras.backend.set_image_data_format('channels_first')

In [0]:
RunLingjuan_Lyu('/content/drive/My Drive/Computer_vision_project/data/LOSO/MHEALTH.npz')

Lingjuan Lyu  et al. 2017 /content/drive/My Drive/Computer_vision_project/data/LOSO/MHEALTH.npz
(2555, 250, 23, 1)
Epoch 1/100
18/18 [==============================] - 1s 45ms/step - loss: 1.6900 - accuracy: 0.5778
Epoch 2/100
18/18 [==============================] - 1s 28ms/step - loss: 0.5230 - accuracy: 0.8848
Epoch 3/100
18/18 [==============================] - 1s 28ms/step - loss: 0.1712 - accuracy: 0.9704
Epoch 4/100
18/18 [==============================] - 1s 28ms/step - loss: 0.0727 - accuracy: 0.9865
Epoch 5/100
18/18 [==============================] - 0s 27ms/step - loss: 0.0398 - accuracy: 0.9952
Epoch 6/100
18/18 [==============================] - 0s 28ms/step - loss: 0.0257 - accuracy: 0.9943
Epoch 7/100
18/18 [==============================] - 1s 28ms/step - loss: 0.0165 - accuracy: 0.9987
Epoch 8/100
18/18 [==============================] - 0s 27ms/step - loss: 0.0091 - accuracy: 1.0000
Epoch 9/100
18/18 [==============================] - 0s 27ms/step - loss: 0.0062 - ac

In [0]:
RunLingjuan_Lyu('/content/drive/My Drive/Computer_vision_project/data/FNOW/MHEALTH.npz')

Lingjuan Lyu  et al. 2017 /content/drive/My Drive/Computer_vision_project/data/FNOW/MHEALTH.npz
(1335, 250, 23, 1)
Epoch 1/100
10/10 [==============================] - 0s 49ms/step - loss: 1.9774 - accuracy: 0.4167
Epoch 2/100
10/10 [==============================] - 0s 26ms/step - loss: 1.0338 - accuracy: 0.8183
Epoch 3/100
10/10 [==============================] - 0s 25ms/step - loss: 0.5116 - accuracy: 0.9092
Epoch 4/100
10/10 [==============================] - 0s 25ms/step - loss: 0.2451 - accuracy: 0.9608
Epoch 5/100
10/10 [==============================] - 0s 26ms/step - loss: 0.1365 - accuracy: 0.9733
Epoch 6/100
10/10 [==============================] - 0s 25ms/step - loss: 0.0804 - accuracy: 0.9833
Epoch 7/100
10/10 [==============================] - 0s 26ms/step - loss: 0.0574 - accuracy: 0.9892
Epoch 8/100
10/10 [==============================] - 0s 26ms/step - loss: 0.0423 - accuracy: 0.9892
Epoch 9/100
10/10 [==============================] - 0s 26ms/step - loss: 0.0352 - ac

In [0]:
RunLingjuan_Lyu('/content/drive/My Drive/Computer_vision_project/data/LOTO/MHEALTH.npz')

Lingjuan Lyu  et al. 2017 /content/drive/My Drive/Computer_vision_project/data/LOTO/MHEALTH.npz
(2555, 250, 23, 1)
Epoch 1/100
18/18 [==============================] - 1s 44ms/step - loss: 1.4817 - accuracy: 0.6420
Epoch 2/100
18/18 [==============================] - 1s 28ms/step - loss: 0.3766 - accuracy: 0.9282
Epoch 3/100
18/18 [==============================] - 0s 28ms/step - loss: 0.1100 - accuracy: 0.9804
Epoch 4/100
18/18 [==============================] - 0s 28ms/step - loss: 0.0495 - accuracy: 0.9913
Epoch 5/100
18/18 [==============================] - 0s 28ms/step - loss: 0.0305 - accuracy: 0.9917
Epoch 6/100
18/18 [==============================] - 0s 28ms/step - loss: 0.0177 - accuracy: 0.9961
Epoch 7/100
18/18 [==============================] - 0s 27ms/step - loss: 0.0095 - accuracy: 0.9996
Epoch 8/100
18/18 [==============================] - 0s 27ms/step - loss: 0.0066 - accuracy: 1.0000
Epoch 9/100
18/18 [==============================] - 0s 27ms/step - loss: 0.0051 - ac

In [0]:
RunLingjuan_Lyu('/content/drive/My Drive/Computer_vision_project/data/SNOW/MHEALTH.npz')

Lingjuan Lyu  et al. 2017 /content/drive/My Drive/Computer_vision_project/data/SNOW/MHEALTH.npz
(2555, 250, 23, 1)
Epoch 1/100
18/18 [==============================] - 1s 46ms/step - loss: 1.5430 - accuracy: 0.6021
Epoch 2/100
18/18 [==============================] - 0s 28ms/step - loss: 0.4814 - accuracy: 0.8872
Epoch 3/100
18/18 [==============================] - 0s 26ms/step - loss: 0.1993 - accuracy: 0.9386
Epoch 4/100
18/18 [==============================] - 0s 27ms/step - loss: 0.1131 - accuracy: 0.9665
Epoch 5/100
18/18 [==============================] - 0s 28ms/step - loss: 0.0706 - accuracy: 0.9791
Epoch 6/100
18/18 [==============================] - 0s 27ms/step - loss: 0.0405 - accuracy: 0.9900
Epoch 7/100
18/18 [==============================] - 0s 27ms/step - loss: 0.0244 - accuracy: 0.9948
Epoch 8/100
18/18 [==============================] - 0s 27ms/step - loss: 0.0181 - accuracy: 0.9970
Epoch 9/100
18/18 [==============================] - 0s 27ms/step - loss: 0.0121 - ac

In [0]:
RunLingjuan_Lyu('/content/drive/My Drive/Computer_vision_project/data/LOSO/USCHAD.npz')

Lingjuan Lyu  et al. 2017 /content/drive/My Drive/Computer_vision_project/data/LOSO/USCHAD.npz
(9824, 500, 6, 1)
Epoch 1/100
70/70 [==============================] - 5s 66ms/step - loss: 1.1953 - accuracy: 0.5943
Epoch 2/100
70/70 [==============================] - 4s 64ms/step - loss: 0.4718 - accuracy: 0.8297
Epoch 3/100
70/70 [==============================] - 4s 64ms/step - loss: 0.3277 - accuracy: 0.8749
Epoch 4/100
70/70 [==============================] - 5s 65ms/step - loss: 0.2582 - accuracy: 0.8979
Epoch 5/100
70/70 [==============================] - 4s 64ms/step - loss: 0.2205 - accuracy: 0.9076
Epoch 6/100
70/70 [==============================] - 4s 63ms/step - loss: 0.2048 - accuracy: 0.9120
Epoch 7/100
70/70 [==============================] - 4s 63ms/step - loss: 0.1820 - accuracy: 0.9205
Epoch 8/100
70/70 [==============================] - 4s 63ms/step - loss: 0.1619 - accuracy: 0.9279
Epoch 9/100
70/70 [==============================] - 4s 63ms/step - loss: 0.1515 - accu

In [0]:
RunLingjuan_Lyu('/content/drive/My Drive/Computer_vision_project/data/LOTO/USCHAD.npz')

Lingjuan Lyu  et al. 2017 /content/drive/My Drive/Computer_vision_project/data/LOTO/USCHAD.npz
(9824, 500, 6, 1)
Epoch 1/100
69/69 [==============================] - 4s 51ms/step - loss: 1.2022 - accuracy: 0.5842
Epoch 2/100
69/69 [==============================] - 4s 51ms/step - loss: 0.4601 - accuracy: 0.8359
Epoch 3/100
69/69 [==============================] - 3s 50ms/step - loss: 0.3193 - accuracy: 0.8802
Epoch 4/100
69/69 [==============================] - 3s 51ms/step - loss: 0.2508 - accuracy: 0.9014
Epoch 5/100
69/69 [==============================] - 3s 51ms/step - loss: 0.2090 - accuracy: 0.9140
Epoch 6/100
69/69 [==============================] - 3s 50ms/step - loss: 0.1871 - accuracy: 0.9183
Epoch 7/100
69/69 [==============================] - 3s 50ms/step - loss: 0.1674 - accuracy: 0.9247
Epoch 8/100
69/69 [==============================] - 4s 51ms/step - loss: 0.1561 - accuracy: 0.9299
Epoch 9/100
69/69 [==============================] - 3s 51ms/step - loss: 0.1454 - accu

In [0]:
RunLingjuan_Lyu('/content/drive/My Drive/Computer_vision_project/data/FNOW/USCHAD.npz')

Lingjuan Lyu  et al. 2017 /content/drive/My Drive/Computer_vision_project/data/FNOW/USCHAD.npz
(5038, 500, 6, 1)
Epoch 1/100
36/36 [==============================] - 2s 53ms/step - loss: 1.6209 - accuracy: 0.4599
Epoch 2/100
36/36 [==============================] - 2s 51ms/step - loss: 0.6978 - accuracy: 0.7640
Epoch 3/100
36/36 [==============================] - 2s 50ms/step - loss: 0.4794 - accuracy: 0.8240
Epoch 4/100
36/36 [==============================] - 2s 51ms/step - loss: 0.3846 - accuracy: 0.8558
Epoch 5/100
36/36 [==============================] - 2s 51ms/step - loss: 0.3325 - accuracy: 0.8770
Epoch 6/100
36/36 [==============================] - 2s 51ms/step - loss: 0.2837 - accuracy: 0.8914
Epoch 7/100
36/36 [==============================] - 2s 50ms/step - loss: 0.2515 - accuracy: 0.9015
Epoch 8/100
36/36 [==============================] - 2s 51ms/step - loss: 0.2301 - accuracy: 0.9130
Epoch 9/100
36/36 [==============================] - 2s 52ms/step - loss: 0.2211 - accu

In [0]:
RunLingjuan_Lyu('/content/drive/My Drive/Computer_vision_project/data/SNOW/USCHAD.npz')

Lingjuan Lyu  et al. 2017 /content/drive/My Drive/Computer_vision_project/data/SNOW/USCHAD.npz
(9824, 500, 6, 1)
Epoch 1/100
70/70 [==============================] - 4s 51ms/step - loss: 1.2599 - accuracy: 0.5711
Epoch 2/100
70/70 [==============================] - 4s 51ms/step - loss: 0.4862 - accuracy: 0.8266
Epoch 3/100
70/70 [==============================] - 4s 51ms/step - loss: 0.3421 - accuracy: 0.8705
Epoch 4/100
70/70 [==============================] - 4s 51ms/step - loss: 0.2688 - accuracy: 0.8949
Epoch 5/100
70/70 [==============================] - 4s 51ms/step - loss: 0.2356 - accuracy: 0.9033
Epoch 6/100
70/70 [==============================] - 4s 51ms/step - loss: 0.2076 - accuracy: 0.9118
Epoch 7/100
70/70 [==============================] - 4s 50ms/step - loss: 0.2016 - accuracy: 0.9144
Epoch 8/100
70/70 [==============================] - 4s 51ms/step - loss: 0.1882 - accuracy: 0.9190
Epoch 9/100
70/70 [==============================] - 4s 51ms/step - loss: 0.1632 - accu

In [0]:
RunLingjuan_Lyu('/content/drive/My Drive/Computer_vision_project/data/LOTO/UTD-MHAD1_1s.npz')

Lingjuan Lyu  et al. 2017 /content/drive/My Drive/Computer_vision_project/data/LOTO/UTD-MHAD1_1s.npz
(3771, 50, 6, 1)
Epoch 1/100
26/26 [==============================] - 0s 12ms/step - loss: 2.8310 - accuracy: 0.1460
Epoch 2/100
26/26 [==============================] - 0s 9ms/step - loss: 2.4130 - accuracy: 0.2696
Epoch 3/100
26/26 [==============================] - 0s 8ms/step - loss: 2.1719 - accuracy: 0.3286
Epoch 4/100
26/26 [==============================] - 0s 9ms/step - loss: 1.9879 - accuracy: 0.3821
Epoch 5/100
26/26 [==============================] - 0s 9ms/step - loss: 1.8440 - accuracy: 0.4265
Epoch 6/100
26/26 [==============================] - 0s 8ms/step - loss: 1.7266 - accuracy: 0.4637
Epoch 7/100
26/26 [==============================] - 0s 9ms/step - loss: 1.6152 - accuracy: 0.4891
Epoch 8/100
26/26 [==============================] - 0s 10ms/step - loss: 1.5041 - accuracy: 0.5221
Epoch 9/100
26/26 [==============================] - 0s 8ms/step - loss: 1.4387 - accura

In [0]:
RunLingjuan_Lyu('/content/drive/My Drive/Computer_vision_project/data/LOSO/UTD-MHAD1_1s.npz')

Lingjuan Lyu  et al. 2017 /content/drive/My Drive/Computer_vision_project/data/LOSO/UTD-MHAD1_1s.npz
(3771, 50, 6, 1)
Epoch 1/100
26/26 [==============================] - 0s 12ms/step - loss: 2.8381 - accuracy: 0.1332
Epoch 2/100
26/26 [==============================] - 0s 9ms/step - loss: 2.4258 - accuracy: 0.2569
Epoch 3/100
26/26 [==============================] - 0s 9ms/step - loss: 2.1652 - accuracy: 0.3319
Epoch 4/100
26/26 [==============================] - 0s 9ms/step - loss: 1.9573 - accuracy: 0.3913
Epoch 5/100
26/26 [==============================] - 0s 9ms/step - loss: 1.8177 - accuracy: 0.4386
Epoch 6/100
26/26 [==============================] - 0s 8ms/step - loss: 1.6968 - accuracy: 0.4803
Epoch 7/100
26/26 [==============================] - 0s 9ms/step - loss: 1.5929 - accuracy: 0.5145
Epoch 8/100
26/26 [==============================] - 0s 9ms/step - loss: 1.4972 - accuracy: 0.5465
Epoch 9/100
26/26 [==============================] - 0s 9ms/step - loss: 1.4131 - accurac

In [0]:
RunLingjuan_Lyu('/content/drive/My Drive/Computer_vision_project/data/FNOW/UTD-MHAD1_1s.npz')

Lingjuan Lyu  et al. 2017 /content/drive/My Drive/Computer_vision_project/data/FNOW/UTD-MHAD1_1s.npz
(2048, 50, 6, 1)
Epoch 1/100
15/15 [==============================] - 0s 12ms/step - loss: 2.9649 - accuracy: 0.1084
Epoch 2/100
15/15 [==============================] - 0s 9ms/step - loss: 2.6735 - accuracy: 0.2000
Epoch 3/100
15/15 [==============================] - 0s 8ms/step - loss: 2.4383 - accuracy: 0.2796
Epoch 4/100
15/15 [==============================] - 0s 8ms/step - loss: 2.2539 - accuracy: 0.3068
Epoch 5/100
15/15 [==============================] - 0s 8ms/step - loss: 2.1027 - accuracy: 0.3433
Epoch 6/100
15/15 [==============================] - 0s 8ms/step - loss: 1.9746 - accuracy: 0.3886
Epoch 7/100
15/15 [==============================] - 0s 8ms/step - loss: 1.8633 - accuracy: 0.4267
Epoch 8/100
15/15 [==============================] - 0s 8ms/step - loss: 1.7577 - accuracy: 0.4578
Epoch 9/100
15/15 [==============================] - 0s 9ms/step - loss: 1.6804 - accurac

In [0]:
RunLingjuan_Lyu('/content/drive/My Drive/Computer_vision_project/data/SNOW/UTD-MHAD1_1s.npz')

Lingjuan Lyu  et al. 2017 /content/drive/My Drive/Computer_vision_project/data/SNOW/UTD-MHAD1_1s.npz
(3771, 50, 6, 1)
Epoch 1/100
27/27 [==============================] - 0s 11ms/step - loss: 2.8237 - accuracy: 0.1494
Epoch 2/100
27/27 [==============================] - 0s 9ms/step - loss: 2.4046 - accuracy: 0.2708
Epoch 3/100
27/27 [==============================] - 0s 9ms/step - loss: 2.1563 - accuracy: 0.3550
Epoch 4/100
27/27 [==============================] - 0s 9ms/step - loss: 1.9584 - accuracy: 0.4019
Epoch 5/100
27/27 [==============================] - 0s 9ms/step - loss: 1.8210 - accuracy: 0.4445
Epoch 6/100
27/27 [==============================] - 0s 9ms/step - loss: 1.6960 - accuracy: 0.4776
Epoch 7/100
27/27 [==============================] - 0s 9ms/step - loss: 1.5862 - accuracy: 0.5103
Epoch 8/100
27/27 [==============================] - 0s 8ms/step - loss: 1.4972 - accuracy: 0.5399
Epoch 9/100
27/27 [==============================] - 0s 9ms/step - loss: 1.4235 - accurac

In [0]:
RunLingjuan_Lyu('/content/drive/My Drive/Computer_vision_project/data/LOTO/UTD-MHAD2_1s.npz')

Lingjuan Lyu  et al. 2017 /content/drive/My Drive/Computer_vision_project/data/LOTO/UTD-MHAD2_1s.npz
(1137, 50, 6, 1)
Epoch 1/100
8/8 [==============================] - 0s 16ms/step - loss: 1.5634 - accuracy: 0.4098
Epoch 2/100
8/8 [==============================] - 0s 8ms/step - loss: 1.2039 - accuracy: 0.5994
Epoch 3/100
8/8 [==============================] - 0s 8ms/step - loss: 1.0094 - accuracy: 0.6432
Epoch 4/100
8/8 [==============================] - 0s 9ms/step - loss: 0.8124 - accuracy: 0.7125
Epoch 5/100
8/8 [==============================] - 0s 9ms/step - loss: 0.6812 - accuracy: 0.7523
Epoch 6/100
8/8 [==============================] - 0s 10ms/step - loss: 0.5864 - accuracy: 0.7859
Epoch 7/100
8/8 [==============================] - 0s 8ms/step - loss: 0.5196 - accuracy: 0.8063
Epoch 8/100
8/8 [==============================] - 0s 8ms/step - loss: 0.4488 - accuracy: 0.8287
Epoch 9/100
8/8 [==============================] - 0s 8ms/step - loss: 0.4040 - accuracy: 0.8359
Epoch 1

In [0]:
RunLingjuan_Lyu('/content/drive/My Drive/Computer_vision_project/data/LOSO/UTD-MHAD2_1s.npz')

Lingjuan Lyu  et al. 2017 /content/drive/My Drive/Computer_vision_project/data/LOSO/UTD-MHAD2_1s.npz
(1137, 50, 6, 1)
Epoch 1/100
8/8 [==============================] - 0s 16ms/step - loss: 1.6382 - accuracy: 0.3029
Epoch 2/100
8/8 [==============================] - 0s 8ms/step - loss: 1.2786 - accuracy: 0.5228
Epoch 3/100
8/8 [==============================] - 0s 8ms/step - loss: 1.0391 - accuracy: 0.6059
Epoch 4/100
8/8 [==============================] - 0s 8ms/step - loss: 0.8707 - accuracy: 0.6667
Epoch 5/100
8/8 [==============================] - 0s 8ms/step - loss: 0.7719 - accuracy: 0.6940
Epoch 6/100
8/8 [==============================] - 0s 8ms/step - loss: 0.6829 - accuracy: 0.7528
Epoch 7/100
8/8 [==============================] - 0s 8ms/step - loss: 0.6285 - accuracy: 0.7842
Epoch 8/100
8/8 [==============================] - 0s 8ms/step - loss: 0.5660 - accuracy: 0.7943
Epoch 9/100
8/8 [==============================] - 0s 8ms/step - loss: 0.5121 - accuracy: 0.8207
Epoch 10

In [0]:
RunLingjuan_Lyu('/content/drive/My Drive/Computer_vision_project/data/FNOW/UTD-MHAD2_1s.npz')

Lingjuan Lyu  et al. 2017 /content/drive/My Drive/Computer_vision_project/data/FNOW/UTD-MHAD2_1s.npz
(616, 50, 6, 1)
Epoch 1/100
5/5 [==============================] - 0s 17ms/step - loss: 1.7366 - accuracy: 0.2333
Epoch 2/100
5/5 [==============================] - 0s 7ms/step - loss: 1.4262 - accuracy: 0.4665
Epoch 3/100
5/5 [==============================] - 0s 7ms/step - loss: 1.2289 - accuracy: 0.5750
Epoch 4/100
5/5 [==============================] - 0s 7ms/step - loss: 1.1056 - accuracy: 0.5877
Epoch 5/100
5/5 [==============================] - 0s 7ms/step - loss: 0.9560 - accuracy: 0.6383
Epoch 6/100
5/5 [==============================] - 0s 7ms/step - loss: 0.8607 - accuracy: 0.6817
Epoch 7/100
5/5 [==============================] - 0s 8ms/step - loss: 0.7642 - accuracy: 0.7052
Epoch 8/100
5/5 [==============================] - 0s 8ms/step - loss: 0.7034 - accuracy: 0.7450
Epoch 9/100
5/5 [==============================] - 0s 7ms/step - loss: 0.6594 - accuracy: 0.7740
Epoch 10/

In [0]:
RunLingjuan_Lyu('/content/drive/My Drive/Computer_vision_project/data/SNOW/UTD-MHAD2_1s.npz')

Lingjuan Lyu  et al. 2017 /content/drive/My Drive/Computer_vision_project/data/SNOW/UTD-MHAD2_1s.npz
(1137, 50, 6, 1)
Epoch 1/100
8/8 [==============================] - 0s 18ms/step - loss: 1.5483 - accuracy: 0.3445
Epoch 2/100
8/8 [==============================] - 0s 8ms/step - loss: 1.1972 - accuracy: 0.5505
Epoch 3/100
8/8 [==============================] - 0s 8ms/step - loss: 0.9805 - accuracy: 0.6438
Epoch 4/100
8/8 [==============================] - 0s 9ms/step - loss: 0.8345 - accuracy: 0.6928
Epoch 5/100
8/8 [==============================] - 0s 8ms/step - loss: 0.7111 - accuracy: 0.7448
Epoch 6/100
8/8 [==============================] - 0s 8ms/step - loss: 0.6294 - accuracy: 0.7664
Epoch 7/100
8/8 [==============================] - 0s 9ms/step - loss: 0.5699 - accuracy: 0.7870
Epoch 8/100
8/8 [==============================] - 0s 8ms/step - loss: 0.4991 - accuracy: 0.8086
Epoch 9/100
8/8 [==============================] - 0s 8ms/step - loss: 0.4485 - accuracy: 0.8312
Epoch 10

In [0]:
RunLingjuan_Lyu('/content/drive/My Drive/Computer_vision_project/data/LOTO/WHARF.npz')

Lingjuan Lyu  et al. 2017 /content/drive/My Drive/Computer_vision_project/data/LOTO/WHARF.npz
(3871, 160, 3, 1)
Epoch 1/100
28/28 [==============================] - 1s 23ms/step - loss: 1.7460 - accuracy: 0.4643
Epoch 2/100
28/28 [==============================] - 1s 18ms/step - loss: 1.2486 - accuracy: 0.5959
Epoch 3/100
28/28 [==============================] - 1s 18ms/step - loss: 1.1064 - accuracy: 0.6414
Epoch 4/100
28/28 [==============================] - 0s 18ms/step - loss: 1.0090 - accuracy: 0.6601
Epoch 5/100
28/28 [==============================] - 0s 18ms/step - loss: 0.9191 - accuracy: 0.6895
Epoch 6/100
28/28 [==============================] - 1s 18ms/step - loss: 0.8682 - accuracy: 0.7045
Epoch 7/100
28/28 [==============================] - 1s 18ms/step - loss: 0.7901 - accuracy: 0.7316
Epoch 8/100
28/28 [==============================] - 1s 18ms/step - loss: 0.7399 - accuracy: 0.7457
Epoch 9/100
28/28 [==============================] - 1s 18ms/step - loss: 0.7099 - accur

In [0]:
RunLingjuan_Lyu('/content/drive/My Drive/Computer_vision_project/data/LOSO/WHARF.npz')

Lingjuan Lyu  et al. 2017 /content/drive/My Drive/Computer_vision_project/data/LOSO/WHARF.npz
(3871, 160, 3, 1)
Epoch 1/100
18/18 [==============================] - 0s 24ms/step - loss: 1.7373 - accuracy: 0.4326
Epoch 2/100
18/18 [==============================] - 0s 17ms/step - loss: 1.3920 - accuracy: 0.5199
Epoch 3/100
18/18 [==============================] - 0s 17ms/step - loss: 1.1812 - accuracy: 0.6254
Epoch 4/100
18/18 [==============================] - 0s 18ms/step - loss: 1.0573 - accuracy: 0.6505
Epoch 5/100
18/18 [==============================] - 0s 18ms/step - loss: 0.9715 - accuracy: 0.6729
Epoch 6/100
18/18 [==============================] - 0s 17ms/step - loss: 0.9111 - accuracy: 0.6779
Epoch 7/100
18/18 [==============================] - 0s 18ms/step - loss: 0.8367 - accuracy: 0.6875
Epoch 8/100
18/18 [==============================] - 0s 18ms/step - loss: 0.7738 - accuracy: 0.7163
Epoch 9/100
18/18 [==============================] - 0s 18ms/step - loss: 0.7521 - accur

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy[0.6738] Recall[0.4390] F1[0.4398] at fold[2]
________________________________________________________________
Epoch 1/100
30/30 [==============================] - 1s 27ms/step - loss: 1.6948 - accuracy: 0.4521
Epoch 2/100
30/30 [==============================] - 1s 18ms/step - loss: 1.2463 - accuracy: 0.5958
Epoch 3/100
30/30 [==============================] - 1s 18ms/step - loss: 1.0569 - accuracy: 0.6456
Epoch 4/100
30/30 [==============================] - 1s 18ms/step - loss: 0.9453 - accuracy: 0.6786
Epoch 5/100
30/30 [==============================] - 1s 18ms/step - loss: 0.8437 - accuracy: 0.6999
Epoch 6/100
30/30 [==============================] - 1s 18ms/step - loss: 0.7848 - accuracy: 0.7255
Epoch 7/100
30/30 [==============================] - 1s 18ms/step - loss: 0.7329 - accuracy: 0.7388
Epoch 8/100
30/30 [==============================] - 1s 18ms/step - loss: 0.6745 - accuracy: 0.7636
Epoch 9/100
30/30 [==============================] - 1s 18ms/step - loss: 0.6228 

In [0]:
RunLingjuan_Lyu('/content/drive/My Drive/Computer_vision_project/data/FNOW/WHARF.npz')

Lingjuan Lyu  et al. 2017 /content/drive/My Drive/Computer_vision_project/data/FNOW/WHARF.npz
(2146, 160, 3, 1)
Epoch 1/100
16/16 [==============================] - 0s 19ms/step - loss: 1.9298 - accuracy: 0.3577
Epoch 2/100
16/16 [==============================] - 0s 17ms/step - loss: 1.4660 - accuracy: 0.5265
Epoch 3/100
16/16 [==============================] - 0s 17ms/step - loss: 1.2624 - accuracy: 0.5836
Epoch 4/100
16/16 [==============================] - 0s 18ms/step - loss: 1.1616 - accuracy: 0.6210
Epoch 5/100
16/16 [==============================] - 0s 18ms/step - loss: 1.0955 - accuracy: 0.6402
Epoch 6/100
16/16 [==============================] - 0s 17ms/step - loss: 1.0664 - accuracy: 0.6563
Epoch 7/100
16/16 [==============================] - 0s 18ms/step - loss: 0.9471 - accuracy: 0.6698
Epoch 8/100
16/16 [==============================] - 0s 17ms/step - loss: 0.9276 - accuracy: 0.6776
Epoch 9/100
16/16 [==============================] - 0s 17ms/step - loss: 0.9131 - accur

In [0]:
RunLingjuan_Lyu('/content/drive/My Drive/Computer_vision_project/data/SNOW/WHARF.npz')

Lingjuan Lyu  et al. 2017 /content/drive/My Drive/Computer_vision_project/data/SNOW/WHARF.npz
(3880, 160, 3, 1)
Epoch 1/100
28/28 [==============================] - 1s 24ms/step - loss: 1.7098 - accuracy: 0.4703
Epoch 2/100
28/28 [==============================] - 0s 18ms/step - loss: 1.1526 - accuracy: 0.6307
Epoch 3/100
28/28 [==============================] - 0s 18ms/step - loss: 0.9961 - accuracy: 0.6643
Epoch 4/100
28/28 [==============================] - 1s 18ms/step - loss: 0.9384 - accuracy: 0.6809
Epoch 5/100
28/28 [==============================] - 1s 18ms/step - loss: 0.8719 - accuracy: 0.7024
Epoch 6/100
28/28 [==============================] - 1s 18ms/step - loss: 0.8197 - accuracy: 0.7220
Epoch 7/100
28/28 [==============================] - 0s 18ms/step - loss: 0.7497 - accuracy: 0.7395
Epoch 8/100
28/28 [==============================] - 1s 18ms/step - loss: 0.7195 - accuracy: 0.7486
Epoch 9/100
28/28 [==============================] - 0s 18ms/step - loss: 0.6833 - accur

In [0]:
RunLingjuan_Lyu('/content/drive/My Drive/Computer_vision_project/data/LOTO/WISDM.npz')

Lingjuan Lyu  et al. 2017 /content/drive/My Drive/Computer_vision_project/data/LOTO/WISDM.npz
(20846, 100, 3, 1)
Epoch 1/100
144/144 [==============================] - 2s 17ms/step - loss: 0.6471 - accuracy: 0.7799
Epoch 2/100
144/144 [==============================] - 2s 16ms/step - loss: 0.3234 - accuracy: 0.8829
Epoch 3/100
144/144 [==============================] - 2s 16ms/step - loss: 0.2235 - accuracy: 0.9214
Epoch 4/100
144/144 [==============================] - 2s 16ms/step - loss: 0.1700 - accuracy: 0.9416
Epoch 5/100
144/144 [==============================] - 2s 16ms/step - loss: 0.1401 - accuracy: 0.9518
Epoch 6/100
144/144 [==============================] - 2s 16ms/step - loss: 0.1137 - accuracy: 0.9600
Epoch 7/100
144/144 [==============================] - 2s 16ms/step - loss: 0.0996 - accuracy: 0.9652
Epoch 8/100
144/144 [==============================] - 2s 16ms/step - loss: 0.0841 - accuracy: 0.9715
Epoch 9/100
144/144 [==============================] - 2s 16ms/step - l

In [12]:
RunLingjuan_Lyu('/content/drive/My Drive/Computer_vision_project/data/LOSO/WISDM.npz')

Lingjuan Lyu  et al. 2017 /content/drive/My Drive/Computer_vision_project/data/LOSO/WISDM.npz
(20846, 100, 3, 1)
Epoch 1/100
158/158 [==============================] - 3s 17ms/step - loss: 0.5639 - accuracy: 0.8040
Epoch 2/100
158/158 [==============================] - 2s 16ms/step - loss: 0.2600 - accuracy: 0.9098
Epoch 3/100
158/158 [==============================] - 2s 16ms/step - loss: 0.1822 - accuracy: 0.9371
Epoch 4/100
158/158 [==============================] - 2s 16ms/step - loss: 0.1429 - accuracy: 0.9502
Epoch 5/100
158/158 [==============================] - 2s 16ms/step - loss: 0.1143 - accuracy: 0.9615
Epoch 6/100
158/158 [==============================] - 3s 16ms/step - loss: 0.0968 - accuracy: 0.9667
Epoch 7/100
158/158 [==============================] - 3s 16ms/step - loss: 0.0792 - accuracy: 0.9725
Epoch 8/100
158/158 [==============================] - 2s 16ms/step - loss: 0.0670 - accuracy: 0.9773
Epoch 9/100
158/158 [==============================] - 3s 16ms/step - l

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy[0.8084] Recall[0.4920] F1[0.5240] at fold[2]
________________________________________________________________
Epoch 1/100
158/158 [==============================] - 3s 17ms/step - loss: 0.6290 - accuracy: 0.7811
Epoch 2/100
158/158 [==============================] - 3s 16ms/step - loss: 0.3114 - accuracy: 0.8879
Epoch 3/100
158/158 [==============================] - 3s 16ms/step - loss: 0.2212 - accuracy: 0.9211
Epoch 4/100
158/158 [==============================] - 3s 16ms/step - loss: 0.1714 - accuracy: 0.9401
Epoch 5/100
158/158 [==============================] - 3s 16ms/step - loss: 0.1420 - accuracy: 0.9506
Epoch 6/100
158/158 [==============================] - 3s 16ms/step - loss: 0.1114 - accuracy: 0.9619
Epoch 7/100
158/158 [==============================] - 3s 16ms/step - loss: 0.0930 - accuracy: 0.9699
Epoch 8/100
158/158 [==============================] - 3s 16ms/step - loss: 0.0775 - accuracy: 0.9748
Epoch 9/100
158/158 [==============================] - 3s 16ms/st

In [12]:
RunLingjuan_Lyu('/content/drive/My Drive/Computer_vision_project/data/FNOW/WISDM.npz')

Lingjuan Lyu  et al. 2017 /content/drive/My Drive/Computer_vision_project/data/FNOW/WISDM.npz
(10516, 100, 3, 1)
Epoch 1/100
74/74 [==============================] - 1s 18ms/step - loss: 0.7185 - accuracy: 0.7504
Epoch 2/100
74/74 [==============================] - 1s 16ms/step - loss: 0.4045 - accuracy: 0.8583
Epoch 3/100
74/74 [==============================] - 1s 16ms/step - loss: 0.3132 - accuracy: 0.8924
Epoch 4/100
74/74 [==============================] - 1s 16ms/step - loss: 0.2469 - accuracy: 0.9136
Epoch 5/100
74/74 [==============================] - 1s 16ms/step - loss: 0.2181 - accuracy: 0.9218
Epoch 6/100
74/74 [==============================] - 1s 16ms/step - loss: 0.1973 - accuracy: 0.9296
Epoch 7/100
74/74 [==============================] - 1s 16ms/step - loss: 0.1717 - accuracy: 0.9390
Epoch 8/100
74/74 [==============================] - 1s 16ms/step - loss: 0.1623 - accuracy: 0.9415
Epoch 9/100
74/74 [==============================] - 1s 16ms/step - loss: 0.1352 - accu

In [13]:
RunLingjuan_Lyu('/content/drive/My Drive/Computer_vision_project/data/SNOW/WISDM.npz')

Lingjuan Lyu  et al. 2017 /content/drive/My Drive/Computer_vision_project/data/SNOW/WISDM.npz
(20846, 100, 3, 1)
Epoch 1/100
147/147 [==============================] - 3s 17ms/step - loss: 0.6182 - accuracy: 0.7906
Epoch 2/100
147/147 [==============================] - 2s 16ms/step - loss: 0.2871 - accuracy: 0.9005
Epoch 3/100
147/147 [==============================] - 2s 16ms/step - loss: 0.1985 - accuracy: 0.9296
Epoch 4/100
147/147 [==============================] - 2s 16ms/step - loss: 0.1500 - accuracy: 0.9454
Epoch 5/100
147/147 [==============================] - 2s 16ms/step - loss: 0.1178 - accuracy: 0.9593
Epoch 6/100
147/147 [==============================] - 2s 16ms/step - loss: 0.0963 - accuracy: 0.9677
Epoch 7/100
147/147 [==============================] - 2s 16ms/step - loss: 0.0818 - accuracy: 0.9717
Epoch 8/100
147/147 [==============================] - 2s 16ms/step - loss: 0.0670 - accuracy: 0.9770
Epoch 9/100
147/147 [==============================] - 2s 16ms/step - l